In [14]:
import numpy as np
import os
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [15]:
train_dir = "./datasets/images/train/"
train_label = "./datasets/labels/train/"
val_dir = "./datasets/images/val"
val_label = "./datasets/labels/val"

In [ ]:
def custom_data_generator(img_dir, train_labels, batch_size=32):
    while True:  
        cropped_images = []
        labels = []
        for file_name in os.listdir(img_dir):
            if file_name.endswith('.jpg'):
                file_txt = file_name[:-4]
                img = cv2.imread(os.path.join(img_dir, file_name))
                img_height, img_width = img.shape[:2]

                label_file = os.path.join(train_labels, f"{file_txt}.txt")
                if not os.path.isfile(label_file):
                    exit(0)

                with open(label_file, "r") as f:
                    content = f.read()

                count = 0
                for line in content.split("\n"):
                    if line == "":
                        continue
                    _, cx, cy, w, h = map(float, line.split(" "))
                    count += 1  

                    xmin = int((cx - w / 2) * img_width)
                    ymin = int((cy - h / 2) * img_height)
                    xmax = int((cx + w / 2) * img_width)
                    ymax = int((cy + h / 2) * img_height)

                    cropped = img[max(ymin, 0):min(ymax, img_height), max(xmin, 0):min(xmax, img_width)]
                    cropped = cv2.resize(cropped, (64, 64)) 
                    cropped_images.append(cropped)

                labels.append(count)

                if len(cropped_images) == batch_size:
                    cropped_images = np.array(cropped_images, dtype="float32") / 255.0
                    batch_labels = np.array(labels, dtype="float32").reshape(-1, 1)  
                    yield cropped_images, batch_labels
                    cropped_images, labels = [], [] 

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(1, activation='linear')  
])

model.compile(optimizer='adam', loss='mae', metrics=['mae'])

train_gen = custom_data_generator(train_dir, train_label, batch_size=32)
val_gen = custom_data_generator(val_dir, val_label, batch_size=32)

steps_per_epoch = (len(os.listdir(train_dir)) + 31) // 32  
validation_steps = (len(os.listdir(val_dir)) + 31) // 32  

model.fit(
    train_gen,
    validation_data=val_gen,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    epochs=10,
    verbose=1
)

loss, mae = model.evaluate(val_gen)
print(f"Validation Loss: {loss}, Validation MAE: {mae}")

test_loss, test_accuracy = model.evaluate(val_gen, verbose=0)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%") 

model.save("human_count_cnn.h5")

KeyboardInterrupt: 